# Mise en place du notebook

## Importation des modules

In [1]:
# Importation de os
import os

# Importation de pickle
import pickle as pk

# Importation de la bibliothèque pandas pour la manipulation de données
import pandas as pd

# Importation des types Dict, List, et Tuple pour une annotation de type plus précise
from typing import Dict, List, Tuple

# Importation de LabelEncoder pour encoder les variables catégoriques
from sklearn.preprocessing import LabelEncoder

# Importation de classification_report pour évaluer les performances du modèle
from sklearn.metrics import classification_report

# Importation de l'objet variable globale
from config import GlobalValue

# Ignorer les avertissements spécifiques de la bibliothèque XGBoost
import warnings
warnings.filterwarnings(action="ignore", module="xgboost")

# Importation de la bibliothèque XGBoost
import xgboost as xgb_

## Mise en place des variables globales

In [2]:
# Chemin du fichier contenant des données simulées d'espèces identifiées
SIMULATED_FILE: str = "./datasets/fakes_species_amplicons.csv"

# Chemin du fichier contenant des données simulées d'espèces non identifiées
UNIDENTIFIED_FILE: str = "./datasets/fakes_unidentified_amplicons.csv"

# Seuils spécifiques à chaque espèce pour classer une prédiction comme positive
SPECIES_THRESHOLDS: Dict[float, float] = {
    0: 0.90,  # Seuil pour Anurofeca_richardsi
    1: 0.90,  # Seuil pour Dermocystidium_salmonis
    2: 0.90,  # Seuil pour Ichthyophonus_hoferi
    3: 0.90,  # Seuil pour Pseudoperkinsus_tapetis
    4: 0.90,  # Seuil pour Psorospermium_haeckelii
    5: 0.90,  # Seuil pour Rhinosporidium_cygnus
    6: 0.90,  # Seuil pour Rhinosporidium_seeberi
    7: 0.90,  # Seuil pour Sphaeroforma_spB7
    8: 0.90,  # Seuil pour Sphaeroforma_spCRG3
    9: 0.90,  # Seuil pour Sphaerothecum_destruens
    10: 0.90  # Seuil pour Anurofeca_richardsi
}

# Listes des barcodes
BARCODES: List[int] = list(range(1, 13))

# Chemin du dossier contenant les séquences vectorisées
DATAS_DIR: str = "./datasets/Sd_testSet_BC"

# Chemin du dossier qui contiendra les résultats
RES_DIR: str = "./results/"

# Chemin du classifieur
CLF_FILE: str = GlobalValue().get_values("clf_file")

# Préfixes des fichiers résultats
PREFIXES: List[str] = ["results", "probabilities", "prediction_distribution", "average_class", "n_reads_species"]

## Définition des fonctions d'analyse des résultats

In [3]:
def analyze_predictions(data: pd.DataFrame, 
                        xgb_model: xgb_.XGBClassifier, 
                        encoder: LabelEncoder) -> Tuple[pd.DataFrame, pd.DataFrame, pd.Series, pd.Series]:
    """
    Analyse les prédictions d'un modèle XGBoost pour un ensemble de données.

    Args:
        data (pd.DataFrame): Données à utiliser.
        xgb_model (XGBClassifier): Modèle XGBoost entraîné.
        encoder (LabelEncoder): Objet LabelEncoder ajusté.

    Returns:
        Tuple[pd.DataFrame, pd.DataFrame, pd.Series, pd.Series]: Les résultats, les probabilités de chaque classe, 
        la distribution des prédictions, la classe moyenne des reads.
    """
    # Prédire les classes
    predictions = xgb_model.predict(X=data)
    predictions = encoder.inverse_transform(y=predictions)

    # Récupérer les prédictions
    results = pd.DataFrame(data={"prediction":predictions})

    # Récupérer les probabilités de chaque classe, pour chaque reads
    probabilities = pd.DataFrame(data=xgb_model.predict_proba(X=data), columns=encoder.classes_)

    # Récupérer la distribution des prédictions
    prediction_distribution = results["prediction"].value_counts()

    # Récupérer la classe moyenne pour l'ensemble des données
    average_class = probabilities.mean().sort_values(ascending=False)

    # Retourner les résultats
    return (results, probabilities, prediction_distribution, average_class)

In [4]:
def process_barcode(number: int, xgb_model: xgb_.XGBClassifier, encoder: LabelEncoder) -> None:
    """
    Traite les données d'un barcode en effectuant des prédictions avec le modèle XGBoost.

    Args:
        number (int): Le numéro du barcode.
        xgb_model (XGBClassifier): Le modèle XGBoost entraîné.
        encoder (LabelEncoder): L'encodeur utilisé.

    Returns:
        None
    """
    # Création du barcode
    barcode = f"0{number}" if number <= 9 else f"{number}"

    # Création du répertoire résultat si inexistant
    if not os.path.exists(path=RES_DIR + f"barcode{barcode}/"):
        os.makedirs(name=RES_DIR + f"barcode{barcode}/")

    # Lecture du fichier de données
    data_file = pd.read_csv(filepath_or_buffer=DATAS_DIR + f"{barcode}.csv")

    # Analyse des prédictions avec le modèle XGBoost et l'encodeur
    results = analyze_predictions(data=data_file, xgb_model=xgb_model, encoder=encoder)

    # Enregistrement des résultats de l'analyse
    for prefix, res in zip(PREFIXES[:-1], results):
        res.to_csv(path_or_buf=RES_DIR + f"barcode{barcode}/{prefix}_{barcode}.csv", index=True)

    # Création du dernier fichier
    with open(file=RES_DIR + f"barcode{barcode}/{PREFIXES[-1]}_{barcode}.csv", mode="w") as tmp:
        # Création des headers
        tmp.write("species,threshold,n_reads_detected\n")

        # Itération pour chaque espèce
        for species_code, threshold in SPECIES_THRESHOLDS.items():
            # Calcul du nombre de prédictions au-dessus du seuil
            prediction_above_threshold = (threshold <= xgb_model.predict_proba(X=data_file)[:, species_code]).astype(int).sum()

            # Conversion du code d'espèce en son nom
            species = encoder.inverse_transform(y=[species_code])[0]

            # Enregistrement du résultat
            tmp.write(f"{species},{threshold:.2f},{prediction_above_threshold}\n")

# Entraînement du modèle

## Chargement des données d'apprentissage

In [5]:
# Charger les données simulées d'espèces identifiées
simulated = pd.read_csv(filepath_or_buffer=SIMULATED_FILE)

# Charger les données simulées d'espèces non identifiées
unindentified = pd.read_csv(filepath_or_buffer=UNIDENTIFIED_FILE)

# Concaténer les deux ensembles de données (espèces identifiées et non identifiées) verticalement (axis=0)
datas = pd.concat(objs=[simulated, unindentified], axis=0)

# Afficher les données concaténées
display(datas)

,0,1,2,3,4,5,6,7,8,9,...,151,152,153,154,155,156,157,158,159,160
0,4,3,4,4,2,1,1,2,2,3,...,0,0,0,0,0,0,0,0,0,Anurofeca_richardsi
1,4,3,4,4,2,1,1,2,2,3,...,3,4,0,0,0,0,0,0,0,Anurofeca_richardsi
2,4,3,4,4,2,1,1,2,2,3,...,0,0,0,0,0,0,0,0,0,Anurofeca_richardsi
3,4,3,4,4,2,1,1,2,2,3,...,4,0,0,0,0,0,0,0,0,Anurofeca_richardsi
4,4,3,4,4,2,1,1,2,2,3,...,3,4,0,0,0,0,0,0,0,Anurofeca_richardsi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,4,3,4,4,2,1,1,2,2,3,...,3,1,3,4,0,0,0,0,0,unidentified
9996,4,3,4,4,2,1,1,2,2,3,...,4,3,1,3,1,3,4,0,0,unidentified
9997,4,3,4,4,2,1,1,2,2,3,...,1,3,1,3,4,0,0,0,0,unidentified
9998,4,3,4,4,2,1,1,2,2,3,...,4,0,0,0,0,0,0,0,0,unidentified


## Encodage des labels et splitting des données en ensemble d'entraînement et de test

In [6]:
# Séparation des caractéristiques (features) et de la cible (target) à partir du DataFrame 'datas'
features, target = datas.iloc[:, :-1], datas.iloc[:, -1]

# Initialisation et ajustement d'un encodeur d'étiquettes (LabelEncoder) sur la cible (target)
encoder = LabelEncoder().fit(y=target)

# Transformation des étiquettes de la cible en valeurs numériques à l'aide de l'encodeur
target = encoder.transform(y=target)

## Entraînement du modèle XGBoost et test sur des données simulées

In [7]:
# Initialisation et ajustement d'un modèle XGBoost Classifier sur l'ensemble d'entraînement
with open(file=CLF_FILE, mode="rb") as model:
    xgb_model: xgb_.XGBClassifier = pk.load(file=model)

# Prédictions sur l'ensemble de test à l'aide du modèle entraîné
y_pred = xgb_model.predict(X=features)

# Affichage du rapport de classification, fournissant des métriques d'évaluation du modèle
print(classification_report(y_true=target, y_pred=y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     10000
           1       1.00      1.00      1.00     10000
           2       1.00      1.00      1.00     10000
           3       1.00      1.00      1.00     10000
           4       1.00      1.00      1.00     10000
           5       0.97      0.99      0.98     10000
           6       0.99      0.97      0.98     10000
           7       1.00      1.00      1.00     10000
           8       1.00      1.00      1.00     10000
           9       1.00      1.00      1.00     10000
          10       1.00      1.00      1.00     10000

    accuracy                           1.00    110000
   macro avg       1.00      1.00      1.00    110000
weighted avg       1.00      1.00      1.00    110000



# Test sur des données réelles

In [8]:
# Analyse pour chaque barcode
for number in BARCODES:
    process_barcode(number=number, xgb_model=xgb_model, encoder=encoder)